### Installing the packages we are finetuning LLama using unsloth

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-y79cuh9z/unsloth_55f0ccc386fa40e1843b4730b46f974a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-y79cuh9z/unsloth_55f0ccc386fa40e1843b4730b46f974a
  Resolved https://github.com/unslothai/unsloth.git to commit 0c8c5ed81e423658ab9ae81eac5aab8d18f5d7af
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install xformers
!pip install triton

### Downloading a large language Model using Unsloth here we are using unsloth/Llama-3.2-1B-Instruct quantized model

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit= load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Parameter efficient Fine Tunning (peft)
- This method allows applying parameter-efficient fine-tuning (PEFT) techniques, such as LoRA (Low-Rank Adaptation), which reduces the number of trainable parameters.
  

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias="none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,)

Unsloth 2024.11.5 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


### Formating the Datset and trainer intialization
- Importing Libraries transformers, trl, unsloth, and datasets libraries are imported to handle tokenization, training, bfloat16 compatibility, and dataset loading.
- EOS_TOKEN is set to the model's end-of-sequence token to mark the end of responses.
- Formatting Function (formatting_prompts_func): This function takes an example from the dataset, formats it using qa_prompt, and appends the EOS_TOKEN at the end. It returns a dictionary with "text" as the key, ready for model training.
- The KisanVaani dataset is loaded, with the train split, and formatted using formatting_prompts_func to match the prompt format.
- SFTTrainer is initialized to fine-tune the model with formatted data.


In [5]:
from transformers import AutoTokenizer
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset



EOS_TOKEN = tokenizer.eos_token

# Define prompt formatting template
qa_prompt = """Below is a question related to agriculture. Write an appropriate response based on the question provided.

### Question:
{}

### Answer:
{}
"""  

# Formatting function for the KisanVaani dataset
def formatting_prompts_func(examples):
    questions = examples["question"]
    answers = examples["answers"]
    texts = []
    
    for question, ans in zip(questions, answers):
        text = qa_prompt.format(question, ans) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Load and format the dataset
dataset = load_dataset("KisanVaani/agriculture-qa-english-only", split="train")
formatted_dataset = dataset.map(formatting_prompts_func, batched=True)

# Initialize the Trainer with the formatted dataset
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=512,  # or your desired sequence length
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


### Training the Model 

In [6]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 22,615 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 11,272,192


Step,Training Loss
1,2.782900
2,3.301500
3,3.268300
4,3.214600
5,2.730100
6,2.180400
7,2.152200
8,2.039400
9,1.884500
10,1.841600


TrainOutput(global_step=60, training_loss=1.6892788231372833, metrics={'train_runtime': 56.1377, 'train_samples_per_second': 8.55, 'train_steps_per_second': 1.069, 'total_flos': 237229060571136.0, 'train_loss': 1.6892788231372833, 'epoch': 0.02122391227449593})

### Testing the Model by giving a Prompt

In [7]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    qa_prompt.format(
        "What are the best practices for crop rotation?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

<|begin_of_text|>Below is a question related to agriculture. Write an appropriate response based on the question provided.

### Question:
What are the best practices for crop rotation?

### Answer:

Crop rotation involves alternating between different crops on the same land to maintain soil fertility, reduce pest and disease buildup, and improve overall crop health.
<|eot_id|>


### saving the model

In [8]:
model.save_pretrained("finetunned_agri_model")
tokenizer.save_pretrained("finetunned_agri_model")

('finetunned_agri_model/tokenizer_config.json',
 'finetunned_agri_model/special_tokens_map.json',
 'finetunned_agri_model/tokenizer.json')